In [ ]:
import sys

sys.path.insert(0, '..')

# Orbital information

In [ ]:
from msrg_vqe import utils

mo_energy, mo_occ = utils.get_orbital_info(
    xyz_file='adamantane.xyz',
    spin=0,
    charge=0,
    basis='sto3g',
)

In [ ]:
mo_energy

In [ ]:
mo_occ

# Make molecule

In [ ]:
import pickle
from msrg_vqe import utils
from msrg_vqe import Molecule
from qiskit_nature.mappers.second_quantization import ParityMapper, JordanWignerMapper, BravyiKitaevMapper

In [ ]:
# If basis set free method
# NOTE: adamantane.zyz.fop is for HOMO-LUMO
with open('adamantane.xyz.fop', 'rb') as fp:
    terms = pickle.load(fp)

custom_second_q_op = utils.openfermion_to_qiskit(terms=terms, num_particles=(1, 1))

In [ ]:
# If basis set free method, uncomment last argument

molecule = Molecule(
    xyz_file='adamantane.xyz',
    active_num_electrons=4,
    active_num_molecular_orbitals=4,
    active_orbitals=[36, 37, 38, 39],
    qubit_mapper=ParityMapper(),
    two_qubit_reduction=True,
    spin=0,
    charge=0,
    basis='sto3g',
#     custom_second_q_op=custom_second_q_op,
)

In [ ]:
molecule.qubit_op

In [ ]:
molecule.ee_property.reference_energy

In [ ]:
molecule.nuclear_repulsion_energy

In [ ]:
molecule.active_space_energy_shift

# RFCI

In [ ]:
from qiskit.algorithms import NumPyMinimumEigensolver

In [ ]:
npe = NumPyMinimumEigensolver()
eigenvalue = npe.compute_minimum_eigenvalue(molecule.qubit_op).eigenvalue

In [ ]:
eigenvalue

In [ ]:
# If not basis set free method, we need to do some processing
eigenvalue + molecule.active_space_energy_shift + molecule.nuclear_repulsion_energy

# VQE

In [ ]:
from qiskit.algorithms.optimizers import *
from qiskit.providers.fake_provider import *
from qiskit.providers.aer import AerSimulator
from qiskit.opflow.gradients import Gradient
from msrg_vqe.ansatz import make_hwe_ansatz, make_esu2_ansatz, make_excitation_preserving_ansatz
from msrg_vqe import VQE

In [ ]:
ansatz = make_hwe_ansatz(molecule, 1)
optimizer = COBYLA()
mitigation_settings = {
    'readout': 'matrix-full', # matrix-full | matrix-tensored | matrix-free,
    'readout-refresh': 5, # recalibrate every 5 min
    'zne': True, # richardson extrapolation
    'stc': False, # state tomography correction at every iteration
    'zne_factory': 'richardson', # (specifies type of zne) richardson | linear
}
backend = AerSimulator.from_backend(FakeLagos())

In [ ]:
vqe = VQE(
    molecule=molecule,
    ansatz=ansatz,
    optimizer=optimizer,
    shots=8192,
    backend=backend, 
    mitigation_settings=mitigation_settings,
    gradient=None,
    debug=1,
    optimization_level=3,
    initial_layout=[0, 1, 2, 3, 4, 5]
)

In [ ]:
from msrg_vqe.lib.adapt_vqe import AdaptVQE

# Note: if want to use AdaptVQE, vqe must be made from UCCSD ansatz
# adapt = AdaptVQE(vqe)
# def cb(*args):
#     print(args)

# adapt.solve(cb)

In [ ]:
parameter_list = []
energy_list = []
i = 1
def callback(d):
    parameter_list.append(d['parameters'])
    energy_list.append(d['energy'])
    global i
    print(i, d)
    i += 1

result = vqe.get_energy(callback=callback)

# State tomography correction as post processing

In [ ]:
# vqe.state_tomography_correction(result.x)